In [1]:
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from skimage import measure
import polyscope as ps
from skimage.measure import marching_cubes

import mlslib as mls


In [2]:

def read_off(file_name: str):
    # Open the file for reading
    with open(file_name, 'r') as file:
        # Read the first line and check if it is 'OFF'
        if file.readline().strip() != 'NOFF':
            raise ValueError('Invalid file format')
        
        # Read the next two lines to get the number of vertices and faces
        num_vertices, num_faces, _ = map(int, file.readline().strip().split())
        
        # Read the vertices
        vertices_normals = [list(map(float, file.readline().strip().split())) for _ in range(num_vertices)]
        vertices, normals = zip(*[[x[:3], x[3:]] for x in vertices_normals])
        
        # Read the faces
        faces = [list(map(int, file.readline().strip().split()))[1:] for _ in range(num_faces)]
        
        # Return the vertices, faces and normals
        return np.array(vertices), np.array(faces), np.array(normals)

Test Data 

In [3]:
from utils import bounding_box_diag ,  generate_grid , vals2colors

def point_cloud_reader(data_path):
        vertices, faces_gt,normals  = read_off(data_path)
        ps.init()
        ps.set_up_dir("z_up") # Fix default camera orientation 
        # gt_mesh = ps.register_surface_mesh("gt_mesh", vertices, faces_gt)
        # gt_mesh.set_enabled(True)

        ps_cloud = ps.register_point_cloud('pts', vertices)
        ps_cloud.add_vector_quantity("Normal vec", normals, radius=0.01, length=0.02, color=(0.2, 0.5, 0.5), enabled=True)

        ps.show() # a window with visualization should be opened.
        return vertices, normals



def point_cloud_csp(vertices , normals):
    bbox_diag = bounding_box_diag(vertices)
    eps = bbox_diag * 0.01 
    new_verts, new_vals = mls.sample_constraints(vertices, normals, eps)
    all_pts = np.concatenate([vertices, new_verts])
    all_vals = np.concatenate([np.zeros(len(vertices)), new_vals])

    ps.register_point_cloud('pos pts', all_pts[all_vals>0])
    ps.register_point_cloud('neg pts', all_pts[all_vals<0])

    return all_pts , all_vals , bbox_diag


def implict_function_eval(all_pts , all_vals , bbox_diag):
    resolution = 30
    grid_pts, coords_matrix = generate_grid(all_pts, resolution)
    local_radius = bbox_diag * 0.1

    ps.register_point_cloud('grid pts', grid_pts, radius=0.001)

    pred_vals = mls.local_predictor(
            grid_pts=grid_pts,
            constr_pts=all_pts,
            constr_vals=all_vals,
            local_radius=local_radius,
            degree=1,
            reg_coef=1)

    colors = vals2colors(pred_vals) # map implicit value to color for visualization
    grid_cloud = ps.register_point_cloud('grid pts', grid_pts, radius=0.001)
    grid_cloud.add_color_quantity("rand colors", colors, enabled=True)
    ps.show()
    return pred_vals , coords_matrix 


def iso_surface(pred_vals , coords_matrix  , resolution):
        verts, faces, _, _ = marching_cubes(pred_vals.reshape([resolution, resolution, resolution]), level=0)
        verts = (coords_matrix[:3, :3] @ verts.T + coords_matrix[:3, 3:4]).T

        pred_mesh = ps.register_surface_mesh("mesh", verts, faces)
        ps.show()

In [4]:
model_names = [ 'cat.off' ]
import time
resolution = 30
for cur_name in model_names:
    vertices, normals = point_cloud_reader(f"./data/{cur_name}")
    all_pts, all_vals, eps = point_cloud_csp(vertices, normals)

    # Start measuring the time before calling implict_function_eval
    start_time = time.time()

    pred_vals, coords_matrix = implict_function_eval(all_pts, all_vals, eps)

    # Stop measuring the time after implict_function_eval has finished
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Output the benchmark results
    print(f"Benchmark for {cur_name}: {elapsed_time} seconds")

    iso_surface(pred_vals , coords_matrix , resolution)
    

Benchmark for bunny-1000.off: 13.930338144302368 seconds
